#### Convert excel file with Patient_Record (clinical endpoint) into tab delimeted file

In [6]:
file_name = 'SampleID_Patient_Record.xlsx'

import os, csv, xlrd
xfile = open(file_name, "r")#open xlsx file 
cfile = open(file_name[:-4]+'tsv', 'w')
wr = csv.writer(cfile, delimiter="\t")# define a writer and delimeter
myfile = xlrd.open_workbook(file_name)# open the xlsx file
mysheet = myfile.sheet_by_index(0)# get a sheet
for rownum in range(mysheet.nrows):# write the rows
    wr.writerow(mysheet.row_values(rownum))
statinfo = os.stat(file_name[:-4]+'tsv')
print(file_name[:-4]+'tsv is created,', 'size =', str(statinfo.st_size)+'b')

#### Make list of values X and corresponding vaues Y

In [7]:
file_name = 'SampleID_Patient_Record.tsv'

import string
from csv import reader
file = open(file_name,'r')#the file is sorted by pru value accending
for line in reader(file, delimiter='\t'):# skip the title line
    break
id_lst = []
pr_lst = []
for rec in reader(file, delimiter='\t'):
    id_lst += [rec[1]]
    pr_lst += [float(rec[5])]
file.close()

#### Run Pearson correlation

In [ ]:
file_name = 'your_cnv_file.tsv'

import scipy, string
from csv import reader
file = open(file_name,'r')
ndx_lst = []
for i in range(len(id_lst)):
    ndx_lst += [0]#index list will retrieve Y data in the order of pr_lst
for rec in reader(file, delimiter='\t'):
    for r in rec:
        if r in id_lst:
            pos_in_rec = rec.index(r)#position of ID in rec
            pos_in_pr = id_lst.index(r)#position of ID in sorted list of pr values
            ndx_lst[pos_in_pr] = pos_in_rec
    break#stop after first line is read
from scipy.stats import pearsonr
Pe_lst = []
Sp_lst = []
newfile = open('P_values.tsv','w')
for rec in reader(file, delimiter='\t'):#read the second row
    cnv_lst = []#list of cnv values
    for n in ndx_lst:
        if ' ' not in rec[n]:#check for multiple empty spaces
            cnv_lst += [float(rec[n])]
        else:
            cnv_lst += [0.0]
    Prsn = scipy.stats.pearsonr(pr_lst,cnv_lst)
    Pe_lst += [[Prsn[1], Prsn[0],rec[4]]]
    Sper = scipy.stats.spearmanr(pr_lst,cnv_lst)#Spearman correlation
    Sp_lst += [[Sper[1], Sper[0], rec[4]]]
    newfile.write(rec[0]+'\t'+rec[1]+'\t'+rec[4]+'\t'+str(Prsn[1])+'\n')
Pe_lst.sort()
for p in Pe_lst:
    if p[0] < 0.05:
        if len(p[2]) < 7:
            print(p[2],'\t\t','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])
        elif len(p[2]) > 14:
            print(p[2],' ','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])
        else: print(p[2],'\t','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])#align long symbols
file.close()

#### Filter genes with at least 23 samples with segment mean > 0.23 or < -0.23

In [ ]:
file_name = 'your_cnv_file.tsv'

import scipy, string
from csv import reader
from scipy.stats import pearsonr

file = open(file_name,'r')
cutoff = 0.23
Pe_lst = []
newfile = open('P_values.tsv','w')
grphfile = open('graph.tsv','w')
for title in reader(file, delimiter='\t'):# skip the title line
    break
for rec in reader(file, delimiter='\t'):#read the second row
    cnv_lst = []#list of cnv values per fragment/gene
    pr_flt_lst = []#filtered pr_lst
    for i in range(5,len(ndx_lst)):# 4 descriptive entries
        if ' ' not in rec[ndx_lst[i]]:#check for multiple empty spaces
            if abs(float(rec[ndx_lst[i]])) > cutoff:
                cnv_lst += [float(rec[ndx_lst[i]])]
                pr_flt_lst += [pr_lst[i]]
    if len(cnv_lst) > 23:
        if rec[4] == 'Gene_name':#make entries for a graph
            for i in range(len(cnv_lst)):
                grphfile.write(str(pr_flt_lst[i]) +'\t'+ str(cnv_lst[i]) +'\n')
            import matplotlib.pyplot as plt
            import numpy as np
            pr = np.array(pr_flt_lst)
            cnv = np.array(cnv_lst)
            plt.scatter(pr, cnv)
            plt.show()
        Prsn = scipy.stats.pearsonr(pr_flt_lst,cnv_lst)
        Pe_lst += [[Prsn[1], Prsn[0],rec[4]]]
        newfile.write(rec[0]+'\t'+rec[1]+'\t'+rec[4]+'\t'+str(Prsn[1])+'\n')
Pe_lst.sort()
print('MULTIPLE TESTS =', len(Pe_lst))
for p in Pe_lst:
    if p[0] < 1:
        if len(p[2]) < 7:
            print(p[2],'\t\t','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])
        elif len(p[2]) > 14:
            print(p[2],' ','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])
        else: print(p[2],'\t','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])#align long symbols

file.close()
newfile.close()
grphfile.close()

#### Filter genes with at least 3 samples with segment mean > 0.3 or < -0.3 but keep all entries for Pearson analysis

In [ ]:
file_name = 'your_cnv_file.tsv'

import scipy, string
from csv import reader
from scipy.stats import pearsonr

file = open(file_name,'r')
cutoff = 0.3
Pe_lst = []
newfile = open('P_values.tsv','w')
for title in reader(file, delimiter='\t'):# skip the title line
    break
for rec in reader(file, delimiter='\t'):#read the second row
    fltr_cnt = 0# count entries that passed the filtering
    cnv_lst = []#list of cnv values per fragment/gene
    pr_flt_lst = []#filtered pr_lst
    for i in range(5,len(ndx_lst)):# 4 descriptive entries
        if ' ' not in rec[ndx_lst[i]]:#check for multiple empty spaces
            if abs(float(rec[ndx_lst[i]])) > cutoff:
                fltr_cnt += 1
            cnv_lst += [float(rec[ndx_lst[i]])]
            pr_flt_lst += [pr_lst[i]]
    if fltr_cnt > 3:
        Prsn = scipy.stats.pearsonr(pr_flt_lst,cnv_lst)
        Pe_lst += [[Prsn[1], Prsn[0],rec[4]]]
        newfile.write(rec[0]+'\t'+rec[1]+'\t'+rec[4]+'\t'+str(Prsn[1])+'\n')
Pe_lst.sort()
for p in Pe_lst:
    if p[0] < 1:
        if len(p[2]) < 7:
            print(p[2],'\t\t','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])
        elif len(p[2]) > 14:
            print(p[2],' ','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])
        else: print(p[2],'\t','R =','%.6f' % p[1],'\t','P-value =','%.6f' % p[0])#align long symbols

file.close()
newfile.close()

#### Spearman correlation

In [ ]:
Sp_lst.sort()
print(Sp_lst[0:11])
for s in Sp_lst:
    if s[0] < 0.05:
        if len(s[2]) < 7:
            print(s[2],'\t\t','R =','%.6f' % s[1],'\t','P-value =','%.6f' % s[0])
        elif len(s[2]) > 14:
            print(s[2],' ','R =','%.6f' % s[1],'\t','P-value =','%.6f' % s[0])
        else: print(s[2],'\t','R =','%.6f' % s[1],'\t','P-value =','%.6f' % s[0])#align long symbols